In [28]:
#!pip install bertopic
#!pip install openpyxl

In [84]:
from pandas import read_excel
data = read_excel("sanatized_data.xlsx",sheet_name = 'sanatized_text')
data

,Unnamed: 0,tweet_id,original_tweets,company_name,sanitized_text_without_url,grams_to_text
0,3,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Google,@sasha_feather @hypatiadotca thank reminder! w...,@sasha_feather @hypatiadotca thank reminder! w...
1,4,1057837349290287104,Text (1/2):\n1. An end to Forced Arbitration i...,Google,text (1/2): 1. end forced_arbitration cases ha...,text (1/2): 1. end forced_arbitration cases ha...
2,6,1057838702330822656,"@hypatiadotca @sasha_feather That’s awesome, w...",Google,"@hypatiadotca @sasha_feather that’s awesome, w...","@hypatiadotca @sasha_feather that’s awesome, w..."
3,20,1057996752735870976,Google employees at the company's U.K. headqua...,Google,employees company's u.k. headquarters staging ...,employees company's u.k. headquarters staging ...
4,33,1058016551796989952,PHOTOS: Worldwide #GoogleWalkout over sexual h...,Google,photos: worldwide #googlewalkout sexual_harass...,photos: worldwide #googlewalkout sexual_harass...
...,...,...,...,...,...,...
938,3130,1465360089947287560,"""The coworker who drugged &amp; raped me, was ...",Apple,"""the coworker drugged raped me, constantly yel...","""the coworker drugged raped me, constantly yel..."
939,3131,1465714221820399618,"""The heartache I had to deal with, constantly ...",Apple,"""the heartache deal with, constantly needing p...","""the heartache deal with, constantly needing p..."
940,3132,1466075237624303616,"""We were told to stop being dramatic and to no...",Apple,"""we told stop dramatic emotional...there endle...","""we told stop dramatic emotional...there endle..."
941,3133,1466440154420240388,"""I [am] scared to speak with an attorney becau...",Apple,"""i [am] scared speak attorney happen job....as...","""i [am] scared speak attorney happen job....as..."


In [85]:
data.head()

,Unnamed: 0,tweet_id,original_tweets,company_name,sanitized_text_without_url,grams_to_text
0,3,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Google,@sasha_feather @hypatiadotca thank reminder! w...,@sasha_feather @hypatiadotca thank reminder! w...
1,4,1057837349290287104,Text (1/2):\n1. An end to Forced Arbitration i...,Google,text (1/2): 1. end forced_arbitration cases ha...,text (1/2): 1. end forced_arbitration cases ha...
2,6,1057838702330822656,"@hypatiadotca @sasha_feather That’s awesome, w...",Google,"@hypatiadotca @sasha_feather that’s awesome, w...","@hypatiadotca @sasha_feather that’s awesome, w..."
3,20,1057996752735870976,Google employees at the company's U.K. headqua...,Google,employees company's u.k. headquarters staging ...,employees company's u.k. headquarters staging ...
4,33,1058016551796989952,PHOTOS: Worldwide #GoogleWalkout over sexual h...,Google,photos: worldwide #googlewalkout sexual_harass...,photos: worldwide #googlewalkout sexual_harass...


In [86]:
import re
#data['grams_to_text'] = data['grams_to_text'].apply(lambda row:  re.sub("@[A-Za-z0-9_]+","", row), 1)
#data['sanitized_text_without_url'] = data['sanitized_text_without_url'].apply(lambda row:  re.sub("@[A-Za-z0-9_]+","", row), 1)
#Removing url and refernces 
data['original_tweets'] = data['original_tweets'].apply(lambda row:  re.sub("@[A-Za-z0-9_]+","", row), 1)
#data['original_tweets'] = data['original_tweets'].apply(lambda row:  re.sub("#[A-Za-z0-9_]+","", row), 1)
data['original_tweets'] = data['original_tweets'].apply(lambda row:  re.sub(r"http\S+", "", row), 1)


In [87]:
data.head()

,Unnamed: 0,tweet_id,original_tweets,company_name,sanitized_text_without_url,grams_to_text
0,3,1057836978954256385,Thank you so much for the reminder! We’re on...,Google,@sasha_feather @hypatiadotca thank reminder! w...,@sasha_feather @hypatiadotca thank reminder! w...
1,4,1057837349290287104,Text (1/2):\n1. An end to Forced Arbitration i...,Google,text (1/2): 1. end forced_arbitration cases ha...,text (1/2): 1. end forced_arbitration cases ha...
2,6,1057838702330822656,"That’s awesome, we’ll use that feature in th...",Google,"@hypatiadotca @sasha_feather that’s awesome, w...","@hypatiadotca @sasha_feather that’s awesome, w..."
3,20,1057996752735870976,Google employees at the company's U.K. headqua...,Google,employees company's u.k. headquarters staging ...,employees company's u.k. headquarters staging ...
4,33,1058016551796989952,PHOTOS: Worldwide #GoogleWalkout over sexual h...,Google,photos: worldwide #googlewalkout sexual_harass...,photos: worldwide #googlewalkout sexual_harass...


In [88]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2",nr_topics=20)

In [89]:
docs = data['original_tweets'].values.tolist()
topics, probs = topic_model.fit_transform(docs)

In [90]:
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings")

In [91]:
topic_model.update_topics(docs, topics=new_topics)

In [92]:
"""
Note that in the above data frame, topic -1 denotes the topic consisting of outlier documents which are typically ignored due to terms having a relatively high prevalence 
across the whole corpus and thus, low specificity toward any cohesive theme or topic.
"""
topic_model.get_topic_info()


,Topic,Count,Name
0,0,29,0_re_you_looking_seen
1,1,74,1_drawalinetableau_68_drew_line
2,2,89,2_google_ai_research_dr
3,3,26,3_compiled_stories_digest_live
4,4,46,4_appletoo_stories_digest_live
5,5,162,5_google_the_to_and
6,6,22,6_githubdropice_notechforice_bad_for
7,7,71,7_ice_notechforice_githubdropice_github
8,8,21,8_vote_off_time_paid
9,9,33,9_zero_emissions_amazon_carbon


In [93]:
topic_model.get_topic(1) 
##You can manually merge topics that u think are similar
## https://maartengr.github.io/BERTopic/getting_started/topicreduction/topicreduction.html


[('drawalinetableau', 0.15710755845549026),
 ('68', 0.15710755845549026),
 ('drew', 0.15549544240095306),
 ('line', 0.15029441765494042),
 ('when', 0.11932175558737439),
 ('children', 0.04621895511091046),
 ('were', 0.04286353268622301),
 ('parents', 0.03649715619409677),
 ('my', 0.03386702716791537),
 ('separated', 0.03303930793779544)]

In [94]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,Thank you so much for the reminder! We’re on...,10,10_the_we_climatestrike_to,the - we - climatestrike - to - climate - our ...,0.967902,False
1,Text (1/2):\n1. An end to Forced Arbitration i...,5,5_google_the_to_and,google - the - to - and - arbitration - sexual...,1.000000,False
2,"That’s awesome, we’ll use that feature in th...",10,10_the_we_climatestrike_to,the - we - climatestrike - to - climate - our ...,0.857776,False
3,Google employees at the company's U.K. headqua...,5,5_google_the_to_and,google - the - to - and - arbitration - sexual...,1.000000,False
4,PHOTOS: Worldwide #GoogleWalkout over sexual h...,5,5_google_the_to_and,google - the - to - and - arbitration - sexual...,1.000000,False
...,...,...,...,...,...,...
938,"""The coworker who drugged &amp; raped me, was ...",4,4_appletoo_stories_digest_live,appletoo - stories - digest - live - is - on -...,0.987930,False
939,"""The heartache I had to deal with, constantly ...",4,4_appletoo_stories_digest_live,appletoo - stories - digest - live - is - on -...,1.000000,False
940,"""We were told to stop being dramatic and to no...",4,4_appletoo_stories_digest_live,appletoo - stories - digest - live - is - on -...,1.000000,False
941,"""I [am] scared to speak with an attorney becau...",4,4_appletoo_stories_digest_live,appletoo - stories - digest - live - is - on -...,1.000000,False


In [95]:
#topic_model.visualize_topics()

In [96]:
#topic_model.visualize_barchart()

In [97]:
#import pprint 
#pprint.pprint(topic_model.get_representative_docs(topic=2))


In [98]:
import pandas as pd
df_topic_document = pd.DataFrame({'topic': topics, 'document': docs})

In [104]:
topic_0 = df_topic_document[df_topic_document.topic == 0]
topic_0

,topic,document
166,0,"📣Our demands, as stated in the petition:📣 1) c..."
169,0,Cease all current and future business with BCF...
170,0,Cease all current and future business with BCF...
171,0,"📣Our demands, as stated in the petition:📣 1) c..."
332,0,20% of Tableau's U.S. based workers signed a l...
333,0,Over 200 employees at Tableau protested the da...
424,0,"First up, a motivating quote from our CEO , pl..."
437,0,"Hi , thank you for sharing your #tableau voice..."
438,0,"Hi ! You may be on the outside now, but still ..."
439,0,"Hi , thank you for and all your #tableau work..."
